<a href="https://colab.research.google.com/github/arjay55/CS284_PA3/blob/main/train_valid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pickle
from torch.utils.tensorboard import SummaryWriter
import time

# from packaging import version
import pandas as pd
import tensorboard as tb

In [3]:
def backup(object, filename):
    with open(filename, 'wb') as f:
        pickle.dump(object, f, pickle.HIGHEST_PROTOCOL)

def reload(persist):
    with open(persist, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        data = pickle.load(f)
    return data

In [4]:
path = "."

In [5]:
class PerImageStandardization(object):
      """
      per_image_whitening function equivalent in PyTorch
      """
      def __call__(self, img):

        adjusted_stddev = torch.max(torch.std(img), 1/torch.sqrt(torch.tensor(torch.numel(img))))

        return (img-torch.mean(img))/adjusted_stddev
        
      def __repr__(self):
        return self.__class__.__name__+'()' 

In [6]:
#Sequence of steps in image transformation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.CenterCrop(size=28),
     PerImageStandardization()])

#train dataset is obtained
trainset = torchvision.datasets.CIFAR10(root=path, train=True,
                                        download=True, transform=transform)

Files already downloaded and verified


In [7]:

#test dataset is obtained
batch_size = 16

#object for distributing data during training and test is defined
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root=path, train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [8]:

class Conv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=0):
        super().__init__()

        #accessible attributes
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, padding_mode='zeros')
        self.batch = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = self.conv(x)
        x = F.relu(self.batch(x))

        return x


## Definition of Neural Network Modules and overall network

In [9]:
class Inception(nn.Module):
    def __init__(self, in_channels, ch1, ch3):
        super().__init__()

        #accessible attributes
        self.ch1 = ch1
        self.ch3 = ch3

        self.convch1 = Conv(in_channels, ch1, 1, 1)
        self.convch3 = Conv(in_channels, ch3, 3, 1, 1)
        
    def forward(self, x):
        ch1features = self.convch1(x)
        ch3features = self.convch3(x)
        x = torch.cat((ch1features,ch3features),1)
        return x


In [10]:
class Downsample(nn.Module):
    def __init__(self, in_channels, ch3):
        super().__init__()

        #accessible attributes
        self.ch3 = ch3

        self.convch3 = Conv(in_channels, ch3, 3, 2)
        self.maxpool = nn.MaxPool2d(3, 2)
        
    def forward(self, x):
        ch3features = self.convch3(x)
        x_maxpool = self.maxpool(x)
        x = torch.cat((ch3features,x_maxpool),1)
        return x


In [11]:
class MiniGoogleNet(nn.Module):
    def __init__(self, in_channels, dropoutval):
        super().__init__()
        
        self.dropoutval = dropoutval

        self.conv_in = Conv(in_channels, 96, 3, 1)

        self.inception1a = Inception(96, 32, 32)
        self.inception1b = Inception(64, 32, 48)
        self.downsample1 = Downsample(32+48, 80)

        self.inception2a = Inception(80+32+48, 112, 48)
        self.inception2b = Inception(112+48, 96, 64)
        self.inception2c = Inception(96+64, 80, 80)
        self.inception2d = Inception(160, 48, 96)
        self.downsample2 = Downsample(48+96, 96)

        self.inception3a = Inception(48+96+96, 176, 160)
        self.inception3b = Inception(176+160, 176, 160)       
        self.meanpool = nn.AdaptiveAvgPool2d(1) #1 is for global mean pooling
        self.dropout = nn.Dropout(self.dropoutval)
        self.fcout = nn.Linear(176+160, 10)

    def forward(self, x):
        x = self.conv_in(x)

        x = self.inception1a(x)
        x = self.inception1b(x)
        x = self.downsample1(x)

        x = self.inception2a(x)
        x = self.inception2b(x)
        x = self.inception2c(x)
        x = self.inception2d(x)
        x = self.downsample2(x)

        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.meanpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fcout(x)
        return x


In [12]:
net = MiniGoogleNet(3, 0.1)

In [13]:
#optimization criteria
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor=1, end_factor=0.01, total_iters=30, last_epoch=-1, verbose=True)

Adjusting learning rate of group 0 to 1.0000e-01.


In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

net.to(device)

cuda:0


MiniGoogleNet(
  (conv_in): Conv(
    (conv): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1))
    (batch): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (inception1a): Inception(
    (convch1): Conv(
      (conv): Conv2d(96, 32, kernel_size=(1, 1), stride=(1, 1))
      (batch): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (convch3): Conv(
      (conv): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (inception1b): Inception(
    (convch1): Conv(
      (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
      (batch): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (convch3): Conv(
      (conv): Conv2d(64, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (batch): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, trac

In [15]:
#tensorboard setup
run_suffix = "Dropout_0p1"
writer = SummaryWriter(comment="_{}".format(run_suffix))

In [16]:
#training proper
interval = 600
start_accuracy = 0
validation_divisor = len(list(enumerate(testloader)))
PATH = 'cifar_net_{}.pth'.format(run_suffix)

counter_train = 0
for epoch in range(80):  # loop over the dataset multiple times

    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        
        if i % interval == interval-1:  
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / interval))
            writer.add_scalar("Loss/train", running_loss / interval, counter_train+1 , walltime = time.time())
            counter_train+=1
            running_loss = 0.0
            
            torch.save(net.state_dict(), 'cifar_net_{}_latest.pth'.format(run_suffix))

    scheduler.step()
    #do validation on every epoch

    correct = 0
    total = 0

    #validation
    running_loss = 0.0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            # images, labels = data
            # calculate outputs by running images through the network 
            outputs = net(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))
    writer.add_scalar("Accuracy/validation", correct / total, epoch+1)
    loss = criterion(outputs, labels)
    validation_loss = running_loss/625
    if correct / total >= start_accuracy:
        torch.save(net.state_dict(), PATH)
        start_accuracy = correct / total
    writer.add_scalar("Loss/validation", validation_loss, epoch+1)
    
writer.flush()
print('Finished Training')

[1,   600] loss: 2.949
[1,  1200] loss: 2.316
[1,  1800] loss: 2.315
[1,  2400] loss: 2.315
[1,  3000] loss: 2.317
Adjusting learning rate of group 0 to 9.6700e-02.
Accuracy of the network on the 10000 test images: 10 %
[2,   600] loss: 2.315
[2,  1200] loss: 2.315
[2,  1800] loss: 2.315
[2,  2400] loss: 2.313
[2,  3000] loss: 2.316
Adjusting learning rate of group 0 to 9.3400e-02.
Accuracy of the network on the 10000 test images: 10 %
[3,   600] loss: 2.314
[3,  1200] loss: 2.313
[3,  1800] loss: 2.314
[3,  2400] loss: 2.312
[3,  3000] loss: 2.313
Adjusting learning rate of group 0 to 9.0100e-02.
Accuracy of the network on the 10000 test images: 10 %
[4,   600] loss: 2.313
[4,  1200] loss: 2.313
[4,  1800] loss: 2.313
[4,  2400] loss: 2.312
[4,  3000] loss: 2.316
Adjusting learning rate of group 0 to 8.6800e-02.
Accuracy of the network on the 10000 test images: 10 %
[5,   600] loss: 2.312
[5,  1200] loss: 2.314
[5,  1800] loss: 2.314
[5,  2400] loss: 2.312
[5,  3000] loss: 2.314
Adjus

Next, let's load back in our saved model (note: saving and re-loading the model
wasn't necessary here, we only did it to illustrate how to do so):



In [19]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # images, labels = data
        # calculate outputs by running images through the network 
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 27 %


In [20]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        # images, labels = data    
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)    
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

  
# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname, 
                                                   accuracy))

Accuracy for class plane is: 43.2 %
Accuracy for class car   is: 27.0 %
Accuracy for class bird  is: 7.2 %
Accuracy for class cat   is: 0.5 %
Accuracy for class deer  is: 0.1 %
Accuracy for class dog   is: 66.3 %
Accuracy for class frog  is: 37.8 %
Accuracy for class horse is: 11.8 %
Accuracy for class ship  is: 43.2 %
Accuracy for class truck is: 46.0 %
